# Laptop Price Prediction in R

## Preprocessing

#### Imports

In [1]:
library(tidyverse)
library(recipes)
library(caret)

── Attaching core tidyverse packages ───────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [2]:
df <- read.csv("../input/laptop_price_prepared.csv")

In [3]:
head(df)

,Company,Product,TypeName,Inches,Cpu,Ram,PrimaryMemSize,PrimaryMemType,SecondaryMemory,OpSys,⋯,Touchscreen,IPS_Panel,RetinaDisplay,CpuBrand,CpuProduct,CpuClockSpeed_GHz,Ram_GB,Gpu_Brand,Gpu_Product,Weight_kg
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,⋯,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<dbl>
1,Apple,MacBook Pro,Ultrabook,13.3,Intel Core i5 2.3GHz,8GB,128GB,SSD,0,macOS,⋯,0,1,1,Intel,i5,5,8,Intel,Iris,1.37
2,Apple,Macbook Air,Ultrabook,13.3,Intel Core i5 1.8GHz,8GB,128GB,Flash Storage,0,macOS,⋯,0,0,0,Intel,i5,5,8,Intel,HD,1.34
3,HP,250 G6,Notebook,15.6,Intel Core i5 7200U 2.5GHz,8GB,256GB,SSD,0,No OS,⋯,0,0,0,Intel,i5,5,8,Intel,HD,1.86
4,Apple,MacBook Pro,Ultrabook,15.4,Intel Core i7 2.7GHz,16GB,512GB,SSD,0,macOS,⋯,0,1,1,Intel,i7,7,16,AMD,Radeon,1.83
5,Apple,MacBook Pro,Ultrabook,13.3,Intel Core i5 3.1GHz,8GB,256GB,SSD,0,macOS,⋯,0,1,1,Intel,i5,5,8,Intel,Iris,1.37
6,Acer,Aspire 3,Notebook,15.6,AMD A9-Series 9420 3GHz,4GB,500GB,HDD,0,Windows 10,⋯,0,0,0,AMD,NA,9,4,AMD,Radeon,2.10


#### Splitting data

In [4]:
y = df$Price_euros

In [5]:
X <- df[, -which(names(df) == "Price_euros")]

#### Dropping columns

In [6]:
cols <- c("Product", "Model", "CpuProduct", "ResolutionWidth", "Inches", "Weight", "Cpu")

X <- X %>%
  select(-one_of(cols))

#### Data types

In [7]:
str(X)

'data.frame':	1295 obs. of  19 variables:
 $ Company          : chr  "Apple" "Apple" "HP" "Apple" ...
 $ TypeName         : chr  "Ultrabook" "Ultrabook" "Notebook" "Ultrabook" ...
 $ Ram              : chr  "8GB" "8GB" "8GB" "16GB" ...
 $ PrimaryMemSize   : chr  "128GB" "128GB" "256GB" "512GB" ...
 $ PrimaryMemType   : chr  "SSD" "Flash Storage" "SSD" "SSD" ...
 $ SecondaryMemory  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ OpSys            : chr  "macOS" "macOS" "No OS" "macOS" ...
 $ Inches_Binned    : int  2 2 4 4 2 4 4 2 3 3 ...
 $ ResolutionHeight : int  1600 900 1080 1800 1600 768 1800 900 1080 1080 ...
 $ DisplayType      : chr  "Retina Display" NA "Full HD" "Retina Display" ...
 $ Touchscreen      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IPS_Panel        : int  1 0 0 1 1 0 1 0 0 1 ...
 $ RetinaDisplay    : int  1 0 0 1 1 0 1 0 0 0 ...
 $ CpuBrand         : chr  "Intel" "Intel" "Intel" "Intel" ...
 $ CpuClockSpeed_GHz: num  5 5 5 7 5 9 7 5 7 5 ...
 $ Ram_GB           : int  8 8 8 16 8 4 16 8 16 

In [8]:
# Specify the preprocessing steps using recipes
recipe <- recipe(~ ., data = X) %>%
  # Step to encode categorical variables
  step_dummy(all_nominal()) %>%
  # Step to scale numerical variables
  step_scale(all_numeric())

# Fit the recipe to your data
processed_X <- prep(recipe, data = X)

# Extract the preprocessed data
processed_X <- bake(processed_X, new_data = NULL)

# View the preprocessed data
print(processed_X)


Warning message:
“! There are new levels in a factor: `NA`.”


# A tibble: 1,295 × 63
   SecondaryMemory Inches_Binned ResolutionHeight Touchscreen IPS_Panel
             <dbl>         <dbl>            <dbl>       <dbl>     <dbl>
 1               0          1.72             5.61           0      2.23
 2               0          1.72             3.16           0      0   
 3               0          3.44             3.79           0      0   
 4               0          3.44             6.32           0      2.23
 5               0          1.72             5.61           0      2.23
 6               0          3.44             2.69           0      0   
 7               0          3.44             6.32           0      2.23
 8               0          1.72             3.16           0      0   
 9               0          2.58             3.79           0      0   
10               0          2.58             3.79           0      2.23
# ℹ 1,285 more rows
# ℹ 58 more variables: RetinaDisplay <dbl>, CpuClockSpeed_GHz <dbl>,
#   Ram_GB <dbl>, Weight

In [9]:
names(processed_X)

[1] "SecondaryMemory"            "Inches_Binned"             
 [3] "ResolutionHeight"           "Touchscreen"               
 [5] "IPS_Panel"                  "RetinaDisplay"             
 [7] "CpuClockSpeed_GHz"          "Ram_GB"                    
 [9] "Weight_kg"                  "Company_Apple"             
[11] "Company_Asus"               "Company_Dell"              
[13] "Company_HP"                 "Company_Lenovo"            
[15] "Company_MSI"                "Company_Other"             
[17] "Company_Toshiba"            "TypeName_Gaming"           
[19] "TypeName_Netbook"           "TypeName_Notebook"         
[21] "TypeName_Ultrabook"         "TypeName_Workstation"      
[23] "Ram_X16GB"                  "Ram_X24GB"                 
[25] "Ram_X2GB"                   "Ram_X32GB"                 
[27] "Ram_X4GB"                   "Ram_X64GB"                 
[29] "Ram_X6GB"                   "Ram_X8GB"                  
[31] "PrimaryMemSize_X128GB"      "PrimaryMemSize_X16GB"      
[33] "PrimaryMemSize_X180GB"      "PrimaryMemSize_X1TB"       
[35] "PrimaryMemSize_X240GB"      "PrimaryMemSize_X256GB"     
[37] "PrimaryMemSize_X2TB"        "PrimaryMemSize_X32GB"      
[39] "PrimaryMemSize_X500GB"      "PrimaryMemSize_X508GB"     
[41] "PrimaryMemSize_X512GB"      "PrimaryMemSize_X64GB"      
[43] "PrimaryMemSize_X8GB"        "PrimaryMemType_HDD"        
[45] "PrimaryMemType_Hybrid"      "PrimaryMemType_SSD"        
[47] "OpSys_Linux"                "OpSys_macOS"               
[49] "OpSys_No.OS"                "OpSys_Windows.10"          
[51] "OpSys_Windows.7"            "DisplayType_Full.HD"       
[53] "DisplayType_Quad.HD."       "DisplayType_Retina.Display"
[55] "CpuBrand_Intel"             "Gpu_Brand_Intel"           
[57] "Gpu_Brand_Nvidia"           "Gpu_Product_HD"            
[59] "Gpu_Product_Iris"           "Gpu_Product_Quadro"        
[61] "Gpu_Product_R17M.M1.70"     "Gpu_Product_Radeon"        
[63] "Gpu_Product_UHD"

#### Test train split

In [12]:
set.seed(123) # for reproducibility
trainIndex <- createDataPartition(y, p = .8, 
                                  list = FALSE, 
                                  times = 1)
X_train <- processed_X[trainIndex, ]
y_train <- y[trainIndex]
X_test  <- processed_X[-trainIndex, ]
y_test  <- y[-trainIndex]

#### Linear Regression

In [13]:
# Perform linear regression
lm_model <- lm(y_train ~ ., data = X_train)

# Summarize the linear regression model
summary(lm_model)

# Predictions
y_pred <- predict(lm_model, newdata = X_test)

# Evaluate the model
mse <- mean((y_pred - y_test)^2)
rmse <- sqrt(mse)
print(paste("Root Mean Squared Error (RMSE):", rmse))


Call:
lm(formula = y_train ~ ., data = X_train)

Residuals:
     Min       1Q   Median       3Q      Max 
-1616.46  -190.46   -41.26   164.72  1615.13 

Coefficients: (10 not defined because of singularities)
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 652.937   1050.800   0.621 0.534560    
SecondaryMemory              27.547     19.805   1.391 0.164701    
Inches_Binned               -58.984     27.949  -2.110 0.035184 *  
ResolutionHeight             89.863    123.876   0.725 0.468439    
Touchscreen                  17.152     23.620   0.726 0.467996    
IPS_Panel                   -10.822     14.102  -0.767 0.443117    
RetinaDisplay               -19.173     56.403  -0.340 0.734015    
CpuClockSpeed_GHz           -44.541     28.188  -1.580 0.114535    
Ram_GB                      127.202    108.793   1.169 0.242723    
Weight_kg                   140.114     27.544   5.087 4.69e-07 ***
Company_Apple               -73.908     85

Warning message in predict.lm(lm_model, newdata = X_test):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”


[1] "Root Mean Squared Error (RMSE): NA"
